# 4 mini graficos. Age1stCode, LearnCode, Stuck y Ed Learn

In [1]:
import pandas as pd

import plotly.express as px

import dash
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc #bootstrap


#crear un dataframe con toda la informacion de la encuesta
df21 = pd.read_csv('../../data/survey_results_public2021.csv', engine="c",
                  usecols= ["MainBranch","Country","US_State","EdLevel", "Age", "Employment",
                   "Age1stCode","LearnCode","YearsCode","YearsCodePro","DevType","OpSys","NEWStuck",
               "ConvertedCompYearly", "LanguageHaveWorkedWith", "LanguageWantToWorkWith","DatabaseHaveWorkedWith",
               "DatabaseWantToWorkWith", "PlatformHaveWorkedWith","PlatformWantToWorkWith",
               "WebframeHaveWorkedWith", "WebframeWantToWorkWith","MiscTechHaveWorkedWith",
               "MiscTechWantToWorkWith", "ToolsTechHaveWorkedWith","ToolsTechWantToWorkWith",
               'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', "ConvertedCompYearly"]) # El indice sera la columna con el ID de la respuesta

#pd.set_option("display.max_rows", None) #permite ver el output completo de la celda


In [2]:
def graficosRespuesta(df, col):

    df=df21[[col]].dropna().copy() #copia df

    df2 = (df.explode(df.columns.tolist())
           .apply(lambda col: col.str.split(';')) 
           .explode(col).groupby(col)
           .agg(responses=(col,'count')) 
           .reset_index())
    
    #eliminar todos los valores que contengan el string "please"
    # ej: Other (please specify)
    df2 = df2[~df2[col].str.contains("please")] 
    df2 = df2[~df2[col].str.contains("not to say")] 
    df2 = df2[~df2[col].str.contains("else")] 
    
    #eliminar la info dentro de parentesis 
    df2[col] = df2[col].str.replace(r"\(.*?\)", "", regex=True) 
    
    #ordenar de mas a menos respuestas
    df2.sort_values(["responses"], ascending=True, inplace = True)

    

    return df2

# 2. Layout

Si no se va a añadir ningun elemento interactivo (ej dropdown o botones) se puede anyadir el gráfico directamente en el layout sin usar callbacks

In [3]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport', #permite ser responsive en movil
                            'content': 'width=device-width, initial-scale=1.0, maximum-scale=1.2, minimum-scale=0.5,'}]
                )

app.layout = html.Div([
    
    dbc.Row([ #dropdown para el primer grafico
            dbc.Col([dcc.Loading([ 
                dcc.Graph(id = 'stuck', figure = px.pie(graficosRespuesta(df21, "NEWStuck"), 
                                                        values='responses', names="NEWStuck",
                                                       title = "What do you do when you get stuck?",
                                                       color_discrete_sequence = px.colors.qualitative.Safe,
                                                       labels={"NEWStuck": "Resource", "responses": "# Responses"})
                         ), #stuck
                ]), #spinner
            ]),#col
            dbc.Col([ dcc.Loading([ #spinner
                dcc.Graph(id = 'LearnCode', figure =  px.pie(graficosRespuesta(df21, "LearnCode"), 
                                                              values='responses', names="LearnCode",
                                                             title = "Learning how to code",
                                                             color_discrete_sequence = px.colors.qualitative.Safe,
                                                             labels={"LearnCode": "Resource", "responses": "# Responses"}
                                                            )
                         ), #learn code
                ]), #spinner
            ])# col
        ]), #row
        
        dbc.Row([ #dropdown para el primer grafico
                    dbc.Col([dcc.Loading([
                        dcc.Graph(id = 'Age1stCode', figure  =
                            (px.bar(graficosRespuesta(df21, "Age1stCode"), y="Age1stCode", x= 'responses',
                                    orientation = "h",text_auto= True, title = "Writing the first line of code", 
                                    color_discrete_sequence = px.colors.qualitative.Safe,
                                   labels={"Age1stCode": "Age start coding", "responses": "# Responses"}))
                                  .update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
                                  .update_layout(yaxis_title=None)
                                  .update_xaxes(tickformat="000")
                                 
                                 ), #Age1stCode 
                        ]), #spinner
                    ]),#col
                    dbc.Col([dcc.Loading([
                        dcc.Graph(id = 'EdLearn', figure  =
                        (px.bar(graficosRespuesta(df21, "EdLevel"), y="EdLevel", x= 'responses',
                                orientation = "h",text_auto= True, title = "Educational level", 
                                color_discrete_sequence = px.colors.qualitative.Safe, 
                               labels={"EdLevel": "Ed Level", "responses": "# Responses"}))
                                  .update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
                                  .update_layout(yaxis_title=None) 

                            ), #edLearn
                        ]), #spinner
                    ]), #col
                     
                ]), #row 
                    

            ]) #div


In [4]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
 * Environment: production
   Use a production WSGI server instead.
   Use a production WSGI server instead.
 * Debug mode: on
 * Debug mode: on
